In [1]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model
import numpy as np
import os
import tensorflow as tf
# os.environ["CUDA_VISIBLE_DEVICES"] = "6"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
from keras.utils import plot_model
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [3]:
base_model = Xception(weights='imagenet')

#【1】创建一个新model, 使得它的输出(outputs)是 VGG19 中任意层的输出(output)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
# print(model.summary())                                 # 打印模型概况

In [4]:
print(base_model.summary())    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

### 提取样例图片的特征

In [57]:
img_path = './elephant.jpg'
img = image.load_img(img_path, target_size=(299, 299)) # 加载图片并resize成224x224

#【3】将图片转化为4d tensor形式
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

#【4】数据预处理
x = preprocess_input(x) #去均值中心化，preprocess_input函数详细功能见注释
model.predict(x)[0]

array([1.2470448e-03, 8.7860107e-02, 1.6193715e-01, ..., 1.5812396e-04,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

### 提取KDEF数据集的特征

In [9]:
import cv2
import imutils
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [42]:
train_path = "./KDEF/"
training_names = os.listdir(train_path)
def imlist(path):
    """
    The function imlist returns all the names of the files in 
    the directory path supplied as argument to the function.
    """
    return [path+'/'+f for f in os.listdir(path)]

In [43]:
# Get all the path to the images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths = []
image_classes = []
class_id = 0
for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imlist(dir)
    image_paths += class_path
    image_classes += [class_id] * len(class_path)
    class_id += 1

In [72]:
xception_features = np.zeros((2940, 2048))
image_paths = sorted(image_paths)
index = 0
for i in range(len(image_paths)):
    if i % 5 >= 2:
        img = image.load_img(image_paths[i], target_size=(299, 299)) # 加载图片并resize成224x224
        #【3】将图片转化为4d tensor形式
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        #【4】数据预处理
        x = preprocess_input(x) #去均值中心化，preprocess_input函数详细功能见注释
        xception_features[index] = model.predict(x)[0]
        index = index + 1

In [75]:
np.savetxt("vgg19.csv", xception_features, delimiter=',')